# Jugando con TOM desde Python

Inspirado en una serie de blogs de David Eldersveld

https://dataveld.com/2020/07/21/python-as-an-external-tool-in-power-bi-desktop-part-2-create-a-pbitool-json-file/

## Crear la cadena de conexión usando los parámetros recibidos desde Power BI

In [ ]:
import os

pbi_server = os.getenv('PBI_SERVER','')
pbi_db = os.getenv('PBI_DB','')

conn_string = f'Provider=MSOLAP;Data Source={pbi_server};Initial Catalog={pbi_db};'

print(f"Server: {pbi_server}")
print(f"Database: {pbi_db}")
print()

## Cargar las bibliotecas .NET

Utiliza la biblioteca pythonnet (https://github.com/pythonnet/pythonnet)

Antes se debe haber instalado en Windows la última versión de la biblioteca AMO (Analysis Management Object)

https://docs.microsoft.com/es-es/analysis-services/client-libraries?view=azure-analysis-services-current


Código copiado de https://github.com/yehoshuadimarsky/python-ssas

In [ ]:
from pathlib import Path

# Python.NET
import clr 

DIR_NET_LIBRARIES = 'C:/Windows/Microsoft.NET/assembly/GAC_MSIL'

# Busca la última versión de la biblioteca TOM, si hay varias versiones instaladas
root = Path(DIR_NET_LIBRARIES)
amo_path = str(
    max((root / 'Microsoft.AnalysisServices.Tabular').iterdir())
    / 'Microsoft.AnalysisServices.Tabular.dll'
)

clr.AddReference('System')
clr.AddReference(amo_path)

import Microsoft.AnalysisServices.Tabular as tom

try:
    type(tom.Server)
    print('Las bibliotecas .NET se cargaron correctamente.')
except NameError:
    print('Error cargando las bibliotecas .NET.')
    

## Conectar al Modelo

In [ ]:
tom_server = tom.Server()
tom_server.Connect(conn_string)

for item in tom_server.Databases:
    print("Base de datos: ", item.Name)
    print("Compatibility Level: ", item.CompatibilityLevel) 
    print("Fecha de creación: ", item.CreatedTimestamp)
    print()

tom_database = tom_server.Databases[0]

## Listar las Tablas, Columnas, Medidas y Relaciones

In [ ]:
from System import Enum

print('TABLAS')
for t in tom_database.Model.Tables:
    print(f'Tabla: {t.Name}')    
    for c in t.Columns:        
        print(f'    Columna: {c.Name}')
        print(f'        Tipo: {Enum.GetName(tom.ColumnType,c.Type)}')
        print(f'        Tipo de dato: {Enum.GetName(tom.DataType,c.DataType)}')
        if c.IsKey:
            print('        Clave')
        if c.IsHidden:
            print('        Oculta')
    for m in t.Measures:
        print(f'    Medida: {m.Name}')
        print(f'        Tipo de dato: {Enum.GetName(tom.DataType,m.DataType)}')
        print(f'        Formato: {m.FormatString}')
        if m.IsHidden:
            print('        Oculta')
    
print('\nRELACIONES')
for r in tom_database.Model.Relationships:
    from_table = r.FromTable.Name
    to_table = r.ToTable.Name
    direction = Enum.GetName(tom.CrossFilteringBehavior,r.CrossFilteringBehavior)
    active_status = 'Activa' if r.IsActive else 'Inactiva'
    print(f'    {from_table} -> {to_table}  ({direction}) ({active_status})')


## El mismo listado anterior pero con formato

Utilizando la biblioteca Rich (https://github.com/willmcgugan/rich)


In [ ]:
from System import Enum
from rich.tree import Tree
from rich.table import Table
from rich import print as rprint

tree = Tree(tom_database.Name)
tree_tables = tree.add('TABLAS')
for t in tom_database.Model.Tables:
    tree_table = tree_tables.add(t.Name)
    table_columns = Table('Columna','Tipo', 'Tipo Dato', 'Clave', 'Oculta')
    for c in t.Columns:        
        column_name = c.Name
        column_type = Enum.GetName(tom.ColumnType,c.Type)
        column_data_type = Enum.GetName(tom.DataType,c.DataType)
        column_key = 'Clave' if c.IsKey else ''
        column_hidden = 'Oculta' if c.IsHidden else ''
        table_columns.add_row(column_name,column_type, column_data_type, column_key, column_hidden)
    tree_table.add(table_columns)        
    if (t.Measures.Count > 0):
        table_measures = Table('Medida','Tipo Dato','Formato','Oculta')
        for m in t.Measures:
            measure_name = m.Name
            measure_data_type = Enum.GetName(tom.DataType,m.DataType)
            measure_format = m.FormatString
            measure_hidden = 'Oculta' if m.IsHidden else ''
            table_measures.add_row(measure_name, measure_data_type, measure_format, measure_hidden)
        tree_table.add(table_measures)
tree_rel = tree.add('RELACIONES')
table_rel = Table('Origen','Destino','Dirección','Estado')
for r in tom_database.Model.Relationships:
    rel_from_table = r.FromTable.Name
    rel_to_table = r.ToTable.Name
    rel_direction = Enum.GetName(tom.CrossFilteringBehavior,r.CrossFilteringBehavior)
    rel_active_status = 'Activa' if r.IsActive else 'Inactiva'
    table_rel.add_row(rel_from_table, rel_to_table, rel_direction, rel_active_status)
tree_rel.add(table_rel)

rprint(tree)

## Listar las Anotaciones del Modelo y de las Tablas

In [ ]:
from rich.tree import Tree
from rich.table import Table
from rich import print as rprint

def get_annotations_table(annotations):
    table = Table('Nombre','Valor', title='Anotaciones')
    for ann in annotations:
        table.add_row(ann.Name,ann.Value)
    return table

tree = Tree(tom_database.Name)
tree.add(get_annotations_table(tom_database.Model.Annotations))
tree_tables = tree.add('TABLAS')
for t in tom_database.Model.Tables:
    tree_table = tree_tables.add(t.Name)
    tree_table.add(get_annotations_table(t.Annotations))

rprint(tree)

## Guardar los listados de Columnas, Medidas y Relaciones en DataFrames

In [ ]:
import pandas as pd
from System import Enum

# Columnas
model_columns =  [
    (
        c.Table.Name,
        c.Name,
        Enum.GetName(tom.DataType,c.DataType),
        'Oculta' if c.IsHidden else 'Visible'
    ) 
for t in tom_database.Model.Tables for c in t.Columns if c.Type != tom.ColumnType.RowNumber]  

df_columns = pd.DataFrame(
    model_columns, 
    columns=['Tabla','Columna','Tipo Dato','Visibilidad']
)
display(df_columns)

# Medidas
model_measures = [
    (
        m.Table.Name,
        m.DisplayFolder,
        m.Name,
        Enum.GetName(tom.DataType,m.DataType),
        m.FormatString,
        'Oculta' if m.IsHidden else 'Visible',
        m.ModifiedTime,                
        m.Expression
    )
for t in tom_database.Model.Tables for m in t.Measures]

df_measures = pd.DataFrame(
    model_measures, 
    columns=['Tabla','Carpeta','Medida','Tipo Dato','Formato','Visibilidad','Modificada','Expresion']
)
display(df_measures)

# Relaciones
model_relationships = [
    (
        r.FromTable.Name,
        r.ToTable.Name,
        Enum.GetName(tom.CrossFilteringBehavior,r.CrossFilteringBehavior),
        'Activa' if r.IsActive else 'Inactiva'
    )
for r in tom_database.Model.Relationships]

df_relationships = pd.DataFrame(
    model_relationships, 
    columns=['Origen','Destino','Direccion','Activa']
)
display(df_relationships)


## Crear un fichero Excel a partir de los DataFrames

In [ ]:
import pandas as pd

EXCEL_FILE = './powerbi_model.xlsx'

with pd.ExcelWriter(EXCEL_FILE) as writer:  
    df_columns.to_excel(writer, sheet_name='Columnas', index=False)
    df_measures.to_excel(writer, sheet_name='Medidas', index=False)
    df_relationships.to_excel(writer, sheet_name='Relaciones', index=False)

print(f'Fichero Excel creado: {EXCEL_FILE}')

## Crear un fichero PDF a partir de los DataFrames
Bibliotecas utilizadas:
- https://github.com/Alexis-benoist/eralchemy para crear el diagrma E/R
- https://www.reportlab.com/dev/install/open_source_installation/ para crear el PDF


In [ ]:
from datetime import datetime 

from eralchemy import render_er
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import Image, Paragraph, SimpleDocTemplate, Table
from reportlab.lib.styles import getSampleStyleSheet

PDF_FILE = 'powerbi_model.pdf'
ER_DIAGRAM_IMAGE = 'powerbi_model.png'

def format_name(name):
    return name.replace(' ','_')

# Generando diagrama ER
model_md = []
model_tables = df_columns['Tabla'].unique()
for t in model_tables:
    name = format_name(t)
    model_md.append(f'[{name}]')
    table_columns = df_columns[df_columns['Tabla'] == t]
    for _,c in table_columns.iterrows():
        if c['Visibilidad'] == 'Oculta':
            continue
        name = format_name(c['Columna'])
        data_type = c['Tipo Dato']
        model_md.append(f'{name} {{label:"{data_type}"}}')
    table_measures = df_measures[df_measures['Tabla'] == t]
    for _,m in table_measures.iterrows():
        if m['Visibilidad'] == 'Oculta':
            continue
        name = format_name(m['Medida'])
        data_type = m['Tipo Dato']
        model_md.append(f'{name} {{label:"{data_type}"}}')
    
for _,r in df_relationships.iterrows():
    from_table = format_name(r['Origen'])
    to_table = format_name(r['Destino'])
    model_md.append(f'{from_table} *--? {to_table}')

render_er(model_md, ER_DIAGRAM_IMAGE)

# Generando PDF
doc = SimpleDocTemplate(PDF_FILE, pagesize=letter)
elements = []

styleSheet = getSampleStyleSheet()

# Insertando fecha y hora
data = [
    ['Fecha:', f'{datetime.now():%Y-%m-%d %H:%M:%S}']
]

t = Table(
    data,
    spaceBefore=0,
    spaceAfter=30,
    style=[
    ('ALIGN', (0, 0), (0, 0),'RIGHT'),
    ('ALIGN', (1, 0), (1, 0),'LEFT')
    ])
elements.append(t)

# Insertando diagrama ER en el PDF
model_image = Image(ER_DIAGRAM_IMAGE)
new_with = doc.width
model_image.drawHeight = new_with * model_image.drawHeight / model_image.drawWidth
model_image.drawWidth = new_with
elements.append(model_image)

# Insertando tabla de Columnas en el PDF
data = [
    [r['Tabla'], r['Columna'], r['Tipo Dato'], r['Visibilidad']]
for _, r in df_columns.iterrows()]

data.insert(0,['COLUMNAS'])
data.insert(1,['Tabla','Columna','Tipo Dato','Visivilidad'])

t = Table(
    data,
    repeatRows=2,
    spaceBefore=30,
    style=[
    ('SPAN', (0, 0), (-1, 0)),
    ('ALIGN', (0, 0), (-1, 0),'CENTER'),
    ('BOX',(0,1),(-1,-1),2,colors.black),
    ('GRID',(0,1),(-1,-1),0.5,colors.black),
    ('BACKGROUND',(0,1),(-1,1),colors.lightgrey)
    ])
elements.append(t)

# Insertando tabla de Medidas en el PDF
data = [
    [r['Tabla'], r['Carpeta'], r['Medida'], r['Formato'], r['Visibilidad'], r['Modificada']]
for _, r in df_measures.iterrows()]

data.insert(0,['MEDIDAS'])
data.insert(1,['Tabla','Carpeta','Medida','Formato','Visibilidad','Modificada'])

t = Table(
    data,
    repeatRows=2,
    spaceBefore=50,
    style=[
    ('SPAN', (0, 0), (-1, 0)),
    ('ALIGN', (0, 0), (-1, 0),'CENTER'),
    ('BOX',(0,1),(-1,-1),2,colors.black),
    ('GRID',(0,1),(-1,-1),0.5,colors.black),
    ('BACKGROUND',(0,1),(-1,1),colors.lightgrey)
    ])
elements.append(t)

# Guardando el PDF
doc.build(elements)

print(f'Fichero PDF creado: {PDF_FILE}')


## Crear Medidas 
Se creará una medida YTD por cada medida que exista en el modelo

Se utilizan anotaciones para guardar con qué herramienta se creó la medida y quien fue el autor

In [ ]:
tom_server.Refresh(True)

for t in tom_database.Model.Tables:
    for m in t.Measures:
        new_measure = tom.Measure()

        new_measure.Name = f'{m.Name} YTD'
        new_measure.Expression = f"TOTALYTD ( [{m.Name}], 'Calendario'[Fecha] )"
        new_measure.DisplayFolder = 'YTD'
        new_measure.Description = f'Calcula {m.Name} desde el inicio del año'
        new_measure.FormatString = m.FormatString

        new_annotation = tom.Annotation()
        new_annotation.Name = 'DevTool'
        new_annotation.Value = 'Python'
        new_measure.Annotations.Add(new_annotation)

        new_annotation = tom.Annotation()
        new_annotation.Name = 'Author'
        new_annotation.Value = 'Nelson'
        new_measure.Annotations.Add(new_annotation)

        m.Table.Measures.Add(new_measure)

        display(f'La medida "{new_measure.Name}" se añadió a la tabla "{new_measure.Table.Name}" y a la carpeta "{new_measure.DisplayFolder}"')

tom_database.Model.SaveChanges()

## Imprimir las medidas que fueron creadas desde Python

In [ ]:
tom_server.Refresh(True)

for t in tom_database.Model.Tables:
    for m in t.Measures:
        dev_tool = m.Annotations.Find('DevTool')
        if dev_tool is not None and dev_tool.Value == 'Python':
            print(f'Medida: [{m.Name}]')
            print(f'Tabla: {m.Table.Name}')
            print(f'Carpeta: {m.DisplayFolder}')
            print('Anotaciones:')
            for a in m.Annotations:
                print(f'    {a.Name} = {a.Value}')
            print()


## Crear un Grupo Calculado
Me he guiado por un ejemplo en C# de Kasper de Jonge https://www.kasperonbi.com/adding-calculation-groups-using-the-tabular-object-model/

El nombre del grupo será "Inteligencia de Tiempo" y tendrá los siguientes items:
- Período Actual
- Mismo Período Mes Anterior
- Mismo Período Año Anterior

In [ ]:
tom_server.Refresh(True)

calculation_group = tom.CalculationGroup()
calculation_group.Precedence = 0

# Todos los items tendrán el mismo formato
FORMAT_STRING = '"#.00"'

# Item Período Actual
calculation_item = tom.CalculationItem()
calculation_item.Name = 'Actual'
calculation_item.Expression = 'SELECTEDMEASURE()'
calculation_item.Ordinal = 1
calculation_item.FormatStringDefinition = tom.FormatStringDefinition()
calculation_item.FormatStringDefinition.Expression = FORMAT_STRING

calculation_group.CalculationItems.Add(calculation_item)

# Item MP Mes Aanterior (Mismo Período Mes Anterior)
calculation_item = tom.CalculationItem()
calculation_item.Name = 'MP Mes Anterior'
calculation_item.Expression = 'CALCULATE(SELECTEDMEASURE(), DATEADD(Calendario[Fecha],-1,month))'
calculation_item.Ordinal = 2
calculation_item.FormatStringDefinition = tom.FormatStringDefinition()
calculation_item.FormatStringDefinition.Expression = FORMAT_STRING

calculation_group.CalculationItems.Add(calculation_item)

# Item MP Año Anterior (Mismo Período Año Anterior)
calculation_item = tom.CalculationItem()
calculation_item.Name = 'MP Año Anterior'
calculation_item.Expression = 'CALCULATE(SELECTEDMEASURE(), DATEADD(Calendario[Fecha],-1,year))'
calculation_item.Ordinal = 3
calculation_item.FormatStringDefinition = tom.FormatStringDefinition()
calculation_item.FormatStringDefinition.Expression = FORMAT_STRING

calculation_group.CalculationItems.Add(calculation_item)

# Hay que crear una Tabla para el Grupo Calculado
cg_table = tom.Table()
cg_table.Name = 'Inteligencia Tiempo'
cg_table.CalculationGroup = calculation_group

# La tabla necesita una partición
cg_partition = tom.Partition()
cg_partition.Name = 'Particion Grupo Calculado Inteligencia Tiempo'
# Existe una fuente de datos específica para los Grupos Calculados
cg_partition.Source = tom.CalculationGroupSource()

cg_table.Partitions.Add(cg_partition)

# Columna de la tabla, requerida por el Grupo Calculado
cg_column = tom.DataColumn()
cg_column.Name = 'Períodos'
cg_column.DataType = tom.DataType.String

cg_table.Columns.Add(cg_column)

# Anotaciones
new_annotation = tom.Annotation()
new_annotation.Name = 'DevTool'
new_annotation.Value = 'Python'
cg_table.Annotations.Add(new_annotation)

new_annotation = tom.Annotation()
new_annotation.Name = 'Author'
new_annotation.Value = 'Nelson'
cg_table.Annotations.Add(new_annotation)

# Adicionando la tabla al modelo
tom_database.Model.Tables.Add(cg_table)
# Para que no haya que actualizar el el modelo manualmente en Power BI
cg_table.RequestRefresh(tom.RefreshType.Full)

# Guardando cambios en el modelo
tom_database.Model.SaveChanges() 
